In [ ]:
#These libraries help to interact with the operating system and the runtime environment respectively
import os
import sys

#Model/Training related libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

#Dataloader libraries
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix

import os
import numpy as np
from PIL import Image
import torchvision   
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision.transforms import Compose 
from torchvision import transforms

from torchsummary import summary
from sklearn.metrics import roc_auc_score

In [ ]:
#Intall Kaggle API and create kaggle directory
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir .kaggle
#This data is used to login  into your Kaggle account
import json
token = {"username":"samruddhi98","key":"db26269bc2e5ae4d4f8456490e50b5a8"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)


In [ ]:
!chmod 600 /content/.kaggle/kaggle.json

!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!kaggle competitions download -c idl-fall21-hw2p2s2-face-verification-slack
# !kaggle competitions download -c idl-fall21-hw2p2s1-face-classification

 90% 140M/156M [00:00<00:00, 192MB/s]
100% 156M/156M [00:00<00:00, 174MB/s]


In [ ]:
!unzip competitions/idl-fall21-hw2p2s2-face-verification-slack/idl-fall21-hw2p2s2-face-verification-slack.zip
# !unzip competitions/idl-fall21-hw2p2s1-face-classification/idl-fall21-hw2p2s1-face-classification.zip

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, data):
        self.data = data
        # self.target_list = target_list
        # self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        i1 = Image.open('verification_data/' + self.data.iloc[index]['img1'])
        i1 = torchvision.transforms.ToTensor()(i1)
        i1 = torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(i1)
        i2 = Image.open('verification_data/' + self.data.iloc[index]['img2'])
        i2 = torchvision.transforms.ToTensor()(i2)
        i2 = torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(i2)
        label = self.data.iloc[index]['label']
        return i1, i2, label

class ImageDataset_test(Dataset):
    def __init__(self, data):
        self.data = data
        # self.target_list = target_list
        # self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        i1 = Image.open('verification_data/' + self.data.iloc[index]['img1'])
        i1 = torchvision.transforms.ToTensor()(i1)
        i1 = torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(i1)
        i2 = Image.open('verification_data/' + self.data.iloc[index]['img2'])
        i2 = torchvision.transforms.ToTensor()(i2)
        i2 = torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(i2)
        # label = self.data.iloc[index]['label']
        return i1, i2

In [ ]:
####### Resnet 18-34 ##########

class SimpleResidualBlock_resnet18(nn.Module):
    def __init__(self, in_channel_size, stride=1):
        super().__init__()
        out_channel_size = int(in_channel_size*stride)
        self.conv1 = nn.Conv2d(in_channel_size, out_channel_size, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(out_channel_size, out_channel_size, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel_size)
        self.bn2 = nn.BatchNorm2d(out_channel_size)
        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(in_channel_size, out_channel_size, kernel_size=1, stride=stride, bias=False)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        shortcut = self.shortcut(x)
        out = self.relu(out + shortcut)
        return out
    
# This has hard-coded hidden feature sizes.
# You can extend this to take in a list of hidden sizes as argument if you want.
class ClassificationNetwork18(nn.Module):
    def __init__(self, in_features, num_classes, feat_dim = 1000):
        super().__init__()
        
        self.layers_34 = nn.Sequential(
            # nn.Conv2d(in_features, 32, kernel_size=3, stride=1, padding=1, bias=True),
            # nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # SimpleResidualBlock_resnet(64),
            SimpleResidualBlock_resnet18(64),
            SimpleResidualBlock_resnet18(64),
            SimpleResidualBlock_resnet18(64,2),
            SimpleResidualBlock_resnet18(128),
            SimpleResidualBlock_resnet18(128),
            SimpleResidualBlock_resnet18(128),
            SimpleResidualBlock_resnet18(128,2),
            SimpleResidualBlock_resnet18(256),
            SimpleResidualBlock_resnet18(256),
            SimpleResidualBlock_resnet18(256),
            SimpleResidualBlock_resnet18(256),
            SimpleResidualBlock_resnet18(256),
            SimpleResidualBlock_resnet18(256,2),
            SimpleResidualBlock_resnet18(512),
            SimpleResidualBlock_resnet18(512),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(), # the above ends up with batch_size x 64 x 1 x 1, flatten to batch_size x 64
        )

        self.layers_18 = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            SimpleResidualBlock_resnet18(64),
            SimpleResidualBlock_resnet18(64,2),
            SimpleResidualBlock_resnet18(128),
            SimpleResidualBlock_resnet18(128,2),
            SimpleResidualBlock_resnet18(256),
            SimpleResidualBlock_resnet18(256,2),
            SimpleResidualBlock_resnet18(512),
            # SimpleResidualBlock_resnet(512),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(), # the above ends up with batch_size x 64 x 1 x 1, flatten to batch_size x 64
        )

        self.linear = nn.Linear(1024, feat_dim)
        nn.init.kaiming_normal(self.linear.weight)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.25)
        self.linear_output = nn.Linear(1024,num_classes)  
        nn.init.kaiming_normal(self.linear_output.weight)   


    def forward(self, x, return_embedding=False):
        embedding_34 = self.layers_34(x) 
        embedding_18 = self.layers_18(x) 
        embedding = torch.cat((embedding_34,embedding_18),1)
        embedding_out = self.relu(self.linear(embedding))
        output = self.linear_output(self.dropout(embedding))
        if return_embedding:
            return embedding_18,output
        else:
            return output   

In [ ]:
closs_weight = 0.1
lr_cent = 0.05
feat_dim = 512
# del network
in_features = 3
num_classes = 4000
learningRate = 0.01
numEpochs = 100
weightDecay = 1e-4
compute_sim = nn.CosineSimilarity(dim=1)
network_18 = ClassificationNetwork18(in_features, num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


In [ ]:
# load model

#### Actual path used for loading various models ###
# epoch = 42 #- gave 83%
# val_acc = 0.8145 #0.7597
# PATH = 'drive/MyDrive/IDL-HW2P2-S1/model-5-CL-SCE_epoch%d_val_acc-%0.04f.pt'%(epoch,val_acc)
# PATH = 'drive/MyDrive/IDL-HW2P2-S2/ensemble_18_34_epoch%d_val_acc-%0.4f.pt'%(epoch,val_acc)
# PATH = 'drive/MyDrive/IDL-HW2P2-S1/model6_Resnet34_epoch%d_val_acc-%0.4f.pt'%(epoch,val_acc)
####################################################

PATH = 'ensemble_model.pt'
print('Loading %s model '%(PATH))
checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
network_18.load_state_dict(checkpoint['model_state_dict'])

Loading drive/MyDrive/IDL-HW2P2-S2/ensemble_18_34_epoch42_val_acc-0.8145.pt model 


<All keys matched successfully>

In [ ]:
data = pd.read_csv('verification_pairs_val.txt',sep=' ', names=['img1','img2','label'])
data_test = pd.read_csv('verification_pairs_test.txt',sep=' ', names=['img1','img2'])

In [ ]:
# dataloader
val_dataset = ImageDataset(data)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, 
                                               shuffle=False, num_workers=4)
test_dataset = ImageDataset_test(data_test)
print(len(test_dataset))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, 
                                               shuffle=False, num_workers=8)

51835


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Validate
# network_34.eval()
network_18.to(device)
network_18.eval()
label_list = []
sim_score = []
print('Length of validation dataloader is: ',len(val_dataloader))

for batch_num, (image1, image2, y) in enumerate(val_dataloader):
    image1, image2 = image1.to(device), image2.to(device)
    label_list.extend(list(y.numpy()))
    emb1_18,o21 = network_18(image1,return_embedding=True)
    emb2_18,o22 = network_18(image2,return_embedding=True)
    similarity = compute_sim(emb1_18, emb2_18).cpu().detach().numpy()
    sim_score.extend(list(similarity))
    if batch_num%100==99:
        print('Processed %d batches'%(batch_num+1))
auc = roc_auc_score(label_list,sim_score)
print('Validation AUC is: {:4f}'.format(auc))

Length of validation dataloader is:  276


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Processed 100 batches
Processed 200 batches
Validation AUC is: 0.926283


In [ ]:
# Test
network_18.to(device)
network_18.eval()
network_res34.eval()
label_list = []
sim_score = []
print('Length of validation dataloader is: ',len(test_dataloader))

for batch_num, (image1, image2) in enumerate(test_dataloader):
    image1, image2 = image1.to(device), image2.to(device)

    emb1_18,o21 = network_18(image1,return_embedding=True)
    emb2_18,o22 = network_18(image2,return_embedding=True)
    similarity = compute_sim(emb1_18, emb2_18).cpu().detach().numpy()
    sim_score.extend(list(similarity))

    #### Tried ensemble method ####
    # emb1_18,out = network_18(image1,return_embedding=True)
    # emb2_18,out = network_18(image2,return_embedding=True)
    # emb1 = torch.cat((0.5*emb1_34,1*emb1_18),1)
    # emb2 = torch.cat((0.5*emb2_34,1*emb2_18),1)
    # similarity = compute_sim(emb1, emb2).cpu().detach().numpy()
    ################################
  
    if batch_num%100==99:
        print('Processed %d batches'%(batch_num+1))


In [ ]:
# creating the submission file

sub_df = pd.read_csv('verification_pairs_test.txt', names=['Id'])
sub_df['Category'] = sim_score
filename = 'submissions_res34-18_epoch42-30.csv'
sub_df.to_csv(filename,index=False)

                                                      Id  Category
0      verification_data/00020839.jpg verification_da...  0.708999
1      verification_data/00002921.jpg verification_da...  0.454112
2      verification_data/00011732.jpg verification_da...  0.322784
3      verification_data/00052778.jpg verification_da...  0.560583
4      verification_data/00053948.jpg verification_da...  0.590733
...                                                  ...       ...
51830  verification_data/00041961.jpg verification_da...  0.372656
51831  verification_data/00060107.jpg verification_da...  0.338923
51832  verification_data/00003205.jpg verification_da...  0.551374
51833  verification_data/00068054.jpg verification_da...  0.655628
51834  verification_data/00046783.jpg verification_da...  0.407868

[51835 rows x 2 columns] 51835


In [ ]:
# Submitting to kaggle
!kaggle competitions submit -c 'idl-fall21-hw2p2s2-face-verification-slack' -f {filename} -m "submission-res34-18_epoch42"

100% 4.01M/4.01M [00:00<00:00, 17.7MB/s]
Successfully submitted to IDL-Fall21-HW2P2S2-Face Verification-Slack